<a href="https://colab.research.google.com/github/gundaminpde/2022/blob/main/Transfer_L_ResNet50_Children_Adults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###어린이와 어른의 사진을 구분하는 분류 모델을 ResNet50을 통해 전이학습한다.

### 앞의 것과 다르게 kaggle에서 직접 자료를 받아 사용하였음.

### 자료는 https://www.kaggle.com/datasets/die9origephit/children-vs-adults-images?select=train 에서 받았음.

from google.colab import drive 
drive.mount('/content/drive/') 



Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalMaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

trans_model = ResNet50(include_top=False, # ResNet50 내부에 원래 있던 마지막 fully connected part를 사용할까? 말까? 원래 True가 default값
                 weights="imagenet",
                 ##input_tensor=None,  ## include_top=True 이면 None
                 input_shape=(200,200,3),    ## include_top=True 이면 None
                 ##pooling=None,
                 classes=1000) #imagenet을 하려면 1000을 값으로 씀

## https://keras.io/api/applications/resnet/ 

## 학생들에게 include_top=False와 True 일 때, 맨마지막 prediction을 보여 주시오.



In [ ]:
trans_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 206, 206, 3)  0           ['input_11[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 100, 100, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [ ]:
trans_model.trainable = False

from tensorflow.keras.layers import Flatten, Dropout

model= Sequential()

model.add(trans_model)
model.add(Flatten())
model.add(Dense(10,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_4 (Flatten)         (None, 100352)            0         
                                                                 
 dense_49 (Dense)            (None, 10)                1003530   
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_50 (Dense)            (None, 2)                 22        
                                                                 
Total params: 24,591,264
Trainable params: 1,003,552
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
pwd

'/'

In [ ]:
train_gen = ImageDataGenerator(
        rescale=1./255,         # 픽셀 값을 0~1 범위로 변환
        rotation_range=40,      # 40도까지 회전
        width_shift_range=0.2,  # 20%까지 좌우 이동
        height_shift_range=0.2, # 20%까지 상하 이동
        shear_range=0.2,        # 20%까지 기울임
        zoom_range=0.2,         # 20%까지 확대
        horizontal_flip=True,   # 좌우 뒤집기
    )


train = train_gen.flow_from_directory(directory='../content/drive/MyDrive/data/train',
                                              target_size=(200, 200),  # 사용할 CNN 모델 입력 사이즈에 맞게 resize
                                              color_mode='rgb',       # 컬러는 rgb, 흑백은 grayscale. 생략하면 컬러로 처리한다
                                              class_mode='categorical', # class_mode='binary',
                                              batch_size=20,
                                              shuffle=True)

valid_gen = ImageDataGenerator(
        rescale=1./255        
    ) 


valid = valid_gen.flow_from_directory(directory='../content/drive/MyDrive/data/test',
                                              target_size=(200, 200),  # 사용할 CNN 모델 입력 사이즈에 맞게 resize
                                              color_mode='rgb',       # 컬러는 rgb, 흑백은 grayscale. 생략하면 컬러로 처리한다
                                              class_mode='categorical', # class_mode='binary',
                                              batch_size=3,
                                              shuffle=True)

hist = model.fit(train, epochs=3, validation_data=valid, verbose=1)



Found 680 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Epoch 1/3
34/34 [==============================] - 126s 4s/step - loss: 1.4394 - accuracy: 0.4868 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/3
34/34 [==============================] - 119s 3s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/3
34/34 [==============================] - 119s 3s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
